---
title: "CPSC 330 Lecture 10: Regression Metrics" 
author: "Varada Kolhatkar"
description: "Metrics for Regression"
description-short: "Ridge, " 
format:
  revealjs:
    html-math-method: mathjax
    embed-resources: true
    slide-number: true
    smaller: true
    center: true
    logo: img/UBC-CS-logo.png
    resources:
      - data/
      - img/  
---

In [ ]:
import os
import sys
import pandas as pd 
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import ConfusionMatrixDisplay 
import mglearn
DATA_DIR = 'data/' 

## Announcements 

- Important information about midterm 1
  - https://piazza.com/class/mekbcze4gyber/post/162
  - **Good news for you: You'll have access to our course notes in the midterm!**
- HW4 was due on Monday, Oct 6th 11:59 pm. 
- HW5 has been released. It's a project-type assignment and you get till Oct 27th to work on it.  


## Focus on the breath!

![](img/inukshuk.jpeg){.nostretch fig-align="center" width="500px"}


## Which Metric Fits Best? {.smaller}

| Scenario | Data Imbalance | Main Concern | Best Metric(s) / Curve |
|-----------|----------------|---------------|------------------------|
| **Email Spam Detection** | 10% spam | Avoid false positives| |
| **Disease Screening** | 1 in 10,000 | Avoid false negatives | |
| **Credit Card Fraud** | 0.1% fraud | Focus on rare positive class | |
| **Customer Churn** | 20% churn | Balance FP & FN | |
| **Sentiment Analysis** | 50/50 balanced | Overall correctness | |
| **Face Recognition** | Balanced pairs | Trade-off FP vs FN | |

## Choosing the Right Metric {.smaller}

| Metric / Plot | When to Use | Why |
|----------------|-------------|-----|
| **Precision, Recall, F1** | When you care about *specific error types* (FP vs FN) or a *fixed threshold*. | Focus on particular tradeoffs. |
| **PR Curve & AP Score** | When the dataset is **highly imbalanced** (rare positives). | Ignores TNs; focuses on positives. |
| **ROC Curve & AUC** | When classes are **moderately imbalanced**. | Measures ranking ability across thresholds. |

### ROC 

How often does the model assign higher scores to positives than to negatives?

	•	Perfect model: AUC = 1.0
	•	Random model: AUC = 0.5

## Questions for you 

- What's the difference between the average precision (AP) score and F1-score? 
- Which model would you pick? 

![](img/pr-curve-which-model.png)

## Questions for you {.smaller}
:::: {.columns}
:::{.column width="60%"}
![](img/roc-baseline.png) 
:::
:::{.column width="40%"}
- What's the AUC of a baseline model? 
:::
::::

[Source](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

## Questions for you {.smaller}

:::: {.columns}
:::{.column width="60%"}
![](img/roc-curve-which-model.png)
:::
:::{.column width="40%"}
- Which model would you pick? 
:::
::::

## Dealing with class imbalance
- Under sampling 
- Oversampling 
- `class weight="balanced"` (preferred method for this course)
- SMOTE

## Handling imbalance by chaning class weights

- We can specify `class_weight="balanced"` (weight each class inversely proportional to their frequency.)


In [ ]:
# This dataset will be loaded using a URL instead of a CSV file
DATA_URL = "https://github.com/firasm/bits/raw/refs/heads/master/creditcard.csv"

cc_df = pd.read_csv(DATA_URL, encoding="latin-1")
# Sorting columns so it is easier to read
cc_df = cc_df[['Class', 'Time', 'Amount'] + cc_df.columns[cc_df.columns.str.startswith('V')].to_list()]

train_df, test_df = train_test_split(cc_df, test_size=0.3, random_state=111)
X_train_big, y_train_big = train_df.drop(columns=["Class", "Time"]), train_df["Class"]
X_test, y_test = test_df.drop(columns=["Class", "Time"]), test_df["Class"]
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_big, y_train_big, test_size=0.3, random_state=123
)
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=num_iter))
pipe_lr.fit(X_train, y_train)

pipe_lr_balanced = make_pipeline(
    StandardScaler(), LogisticRegression(max_iter=1000, class_weight="balanced")
)
pipe_lr_balanced.fit(X_train, y_train)

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ConfusionMatrixDisplay.from_estimator(
    pipe_lr,
    X_valid,
    y_valid,
    values_format="d",
    ax=ax[0],
    colorbar=False
)
ax[0].set_title("LR")

ConfusionMatrixDisplay.from_estimator(
    pipe_lr_balanced,
    X_valid,
    y_valid,
    values_format="d",
    ax=ax[1], 
    colorbar=False
)
ax[1].set_title("balanced")

plt.tight_layout()
plt.show()

::: {.notes}
- This sets the weights so that the classes are "equal".
- We have reduced false negatives but we have many more false positives now ...
  - Decreases false negatives, which improves recall.
  - Increases false positives, which might lower precision.
:::

# [Regression metrics class demo](https://github.com/UBC-CS/cpsc330-2025W1/blob/main/lectures/102-Varada-lectures/class_demos/demo_10-regression-metrics.ipynb)

## Ridge and RidgeCV
- **Ridge Regression**: `alpha` hyperparameter controls model complexity.
- **RidgeCV**: Ridge regression with built-in cross-validation to find the optimal `alpha`.

## `alpha` hyperparameter
- **Role of `alpha`**:
  - Controls model complexity
  - Higher `alpha`: Simpler model, smaller coefficients.
  - Lower `alpha`: Complex model, larger coefficients.

## Regression metrics: MSE, RMSE, MAPE

- **Mean Squared Error (MSE)**: Average of the squares of the errors.
- **Root Mean Squared Error (RMSE)**: Square root of MSE, same units as the target variable.
- **Mean Absolute Percentage Error (MAPE)**: Average of the absolute percentage errors.

## Applying log transformation to the targets

- Suitable when the target has a wide range and spans several orders of magnitude 
  - Example: counts data such as social media likes or price data
- Helps manage skewed data, making patterns more apparent and regression models more effective.
- `TransformedTargetRegressor`
  - Wraps a regression model and applies a transformation to the target values.


## iClicker Exercise 10.1

**Select all of the following statements which are TRUE.**

- (A) Price per square foot would be a good feature to add in our `X`. 
- (B) The `alpha` hyperparameter of `Ridge` has similar interpretation of `C` hyperparameter of `LogisticRegression`; higher `alpha` means more complex model. 
- (C) In `Ridge`, smaller alpha means bigger coefficients whereas bigger alpha means smaller coefficients.  


## iClicker Exercise 10.2

**Select all of the following statements which are TRUE.**

- (A) We can use still use precision and recall for regression problems but now we have other metrics we can use as well.
- (B) In `sklearn` for regression problems, using `r2_score()` and `.score()` (with default values) will produce the same results.
- (C) RMSE is always going to be non-negative.
- (D) MSE does not directly provide the information about whether the model is underpredicting or overpredicting.
- (E) We can pass multiple scoring metrics to `GridSearchCV` or `RandomizedSearchCV` for regression as well as classification problems. 